### Training Neural Networks

### Losses in Pytorch
Using the softmax function to predict class probabilities. With a softmax output, using cross-entropy as the loss. To actually calculate the loss, first define the criterion then pass in the output of your network and the correct labels.

In [3]:
#softmax gives you probabilities which will often be very close to zero or one but floating-point numbers
#can't accurately represent values near zero or one, we use log-probabilities. 
#This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import helper

# Define transform to normalize the data
# mean, std = 0.5, 0.5
tranform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])

#Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=tranform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

image, label = next(iter(trainloader))
helper.imshow(image[0,:])
trainset

Dataset MNIST
    Number of datapoints: 60000
    Root location: /Users/taniaincio/.pytorch/MNIST_data/
    Split: Train

In [5]:
# Build a feed-fordward network
n_input = 784
n_hiddens = [128, 64]
n_output = 10

model = nn.Sequential(nn.Linear(n_input, n_hiddens[0]),
                     nn.ReLU(),
                     nn.Linear(n_hiddens[0], n_hiddens[1]),
                     nn.ReLU(),
                     nn.Linear(n_hiddens[1], n_output),
                     nn.Softmax(dim = 1))

# Define the loss
criterion = nn.CrossEntropyLoss()

# Get data
images, labels = next(iter(trainloader))
# Flatten images
images = images.view(images.shape[0], -1)

# Forward pass, get our logits
logits = model(images)
# Calculate the loss with the logits and the labels
loss = criterion(logits, labels)
print(loss)


tensor(2.3052, grad_fn=<NllLossBackward>)


In [6]:
# TODO: it's more convenient to build the model with log-softmax output and using nn.NLLLLoss.
# USE LogSoftmax and nn.NLLLos togheter
# TODO:  nn.CrossEntropyLoss combines two functions nn.NLLLoss and LogSoftmax
# it is preferable to use nn.NLLLoss since it requires output in softmax and we can easily get probabilities by using exponential
model = nn.Sequential(nn.Linear(784, 128),
                     nn.ReLU(),
                     nn.Linear(128, 64),
                     nn.ReLU(),
                     nn.Linear(64, 10),
                     nn.LogSoftmax(dim = 1))

# TODO: Define the loss
criterion = nn.NLLLoss()

### Run this to check your work
# Get our data
images, labels = next(iter(trainloader))
# Flatten images
images = images.view(images.shape[0], -1)

# Forward pass, get our logits
logits = model(images)
# Calculate the loss with the logits and the labels
loss = criterion(logits, labels)

print(loss)

tensor(2.2865, grad_fn=<NllLossBackward>)


## Autograd

Module that automatically calculating the gradients of tensors

In [7]:
# Requires_grad = True on a tensor, x.requires_grad_(True), torch.no_grad(), torch.set_grad_enabled(True|False)
x = torch.zeros(1, requires_grad = True)
with torch.no_grad():
    y = x * 2
y.requires_grad

False

In [8]:
# Example getting gradients with z.backward()
x = torch.randn(2, 2, requires_grad = True)
print(x)

y = x**2
print(y)

## grad_fn shows the function that generated this variable
print(y.grad_fn)

z = y.mean()
print(z)


z.backward()
print(x.grad)

tensor([[-1.4658,  1.0586],
        [ 1.8547,  1.3425]], requires_grad=True)
tensor([[2.1486, 1.1207],
        [3.4399, 1.8023]], grad_fn=<PowBackward0>)
tensor(2.1279, grad_fn=<MeanBackward0>)
tensor([[-0.7329,  0.5293],
        [ 0.9274,  0.6713]])


In [9]:
## For training need the gradients of the cost with respect to the weights.
## NOTE: With PyTorch, run data forward through the network to calculate the loss, then, go backwards to calculate the gradients with respect to the loss. 
## Once we have the gradients we can make a gradient descent step.

### Loss and Autograd 

In [14]:
##
n_input = 784
n_hiddens = [128, 64]
n_output = 10
model = nn.Sequential(nn.Linear(n_input, n_hiddens[0]),
                      nn.ReLU(),
                      nn.Linear(n_hiddens[0], n_hiddens[1]),
                      nn.ReLU(),
                      nn.Linear(n_hiddens[1], n_output),
                      nn.LogSoftmax(dim = 1))

criterion = nn.NLLLoss()

images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

logits = model(images)
loss = criterion(logits, labels)
print('Before backward pass: \n', model[0].weight.grad)

loss.backward()

print('After backward pass: \n', model[0].weight.grad)

Before backward pass: 
 None
After backward pass: 
 tensor([[0.0002, 0.0002, 0.0002,  ..., 0.0002, 0.0002, 0.0002],
        [0.0007, 0.0007, 0.0007,  ..., 0.0007, 0.0007, 0.0007],
        [0.0005, 0.0005, 0.0005,  ..., 0.0005, 0.0005, 0.0005],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0012, 0.0012, 0.0012,  ..., 0.0012, 0.0012, 0.0012],
        [0.0012, 0.0012, 0.0012,  ..., 0.0012, 0.0012, 0.0012]])
